# AI/LLM HandsOn for Beginners

## 01. AI SQL
![IMAGE](https://lh3.googleusercontent.com/pw/AP1GczOgvFatwYB0QoHXNYFdRDtD57XNZqC7rCjkQtR0eMD0mX1dCNTMzfu6oNpX293tHPTEQtGuScjzzjXTlUe5ScqQnTppun1dTUXVa4H31qLBJaySa8lyoLDbJakqEMW3xLG47yl3ClPSh1UU4_i26Akh=w1700-h956-s-no-gm?authuser=0)

![IMAGE](https://lh3.googleusercontent.com/pw/AP1GczOXh3TgEdD5vrl1cDY3RJ3lsemM3B9BdLQoIKIu_XSJPY5iJhcMewal75jq0ddx9x_LD3judc-amBpueb0kvnW4fK9ikqIlDS7FAiYRpSGZoO8t0tvtEQYf7Ng_B2FZk6M7JoaKGG226e6eQw3k_KsA=w960-h540-s-no-gm?authuser=0)

In [ ]:
import streamlit as st
import pandas as pd
import altair as alt

from snowflake.snowpark.context import get_active_session
session = get_active_session()

### 1-1-1 : 汎用関数 (AI_COMPLETE) SQL版
![IMAGE](https://lh3.googleusercontent.com/pw/AP1GczM-zFWUrEyHtZehRNgvwdJdW4H8tlEC04Hdbt2i0ipojb4X_2OAH3e7t2wPKIkft5lB5P-hR-us2bKp1Cma_bGTb7vMWJdg8DHZceZo0xikWuylg4kOPq6FyqUA4katVBJ8YGwRpj6BKfClC6p1jGKo=w3366-h1888-s-no-gm?authuser=0)
- [Snowflake Document - AI_COMPLETE(Single string)](https://docs.snowflake.com/en/sql-reference/functions/ai_complete-single-string "AI_COMPLETE (Single string)")
- [Snowflake Document - 各CSP/Regionごとの利用可能LLM一覧](https://docs.snowflake.com/en/user-guide/snowflake-cortex/aisql#availability "Availability")




In [ ]:
-- SELECT AI_COMPLETE(
--    <model>, <prompt> [ , <model_parameters>, <response_format>, <show_details> ] );

In [ ]:
SET input_model = 'mistral-large2';
SET input_prompt = '大規模言語モデルとは何ですか?';

SELECT AI_COMPLETE(
    $input_model, $input_prompt) as response;

In [ ]:
-- AI_COMPLETE関数を使ってみましょう

### 1-1-2 : 汎用関数 (AI_COMPLETE) Python版
- [Snowflake Document - snowflake.snowpark.functions.ai_complete](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/snowpark/api/snowflake.snowpark.functions.ai_complete "snowflake.snowpark.functions.ai_complete")


In [ ]:
from snowflake.snowpark.functions import ai_complete


input_model = 'mistral-large2'
input_pormpt = '大規模言語モデルとは何ですか?'

response = session.range(1).select(
    ai_complete(
        model = input_model,
        prompt = input_pormpt
    ).alias("response")
).collect()[0][0]

st.markdown(response.strip('"').replace('\\n', '\n'))

### 1-1-3: AI_COMPLETEをテーブルに適用
- サンプルのデータベース/テーブルを作成
- サンプルレコードを作成
- AI_COMPLETE関数を実行

In [ ]:
-- サンプルのデータベースを作成
CREATE OR REPLACE DATABASE LLM_FOR_BEGINNERS_DB;

-- サンプルテーブルを作成
CREATE
OR REPLACE TABLE LLM_FOR_BEGINNERS_DB.PUBLIC.INSURANCE_REVIEWS (
  REVIEW_ID DECIMAL(38, 0) AUTOINCREMENT,
  CUSTOMER_ID VARCHAR(10),
  SALES_REP_ID VARCHAR(10),
  REVIEW_DATE DATE,
  RATING DECIMAL(1),
  REVIEW_COMMENT VARCHAR(1000)
);

In [ ]:
/* サンプルデータの生成（100レコード） */
INSERT INTO
  LLM_FOR_BEGINNERS_DB.PUBLIC.INSURANCE_REVIEWS (
    CUSTOMER_ID,
    SALES_REP_ID,
    REVIEW_DATE,
    RATING,
    REVIEW_COMMENT
  ) WITH SAMPLE_DATA AS (
    SELECT
      'C' || LPAD (
        ROW_NUMBER() OVER (
          ORDER BY
            SEQ4 ()
        ),
        4,
        '0'
      ) AS CUSTOMER_ID,
      'S' || LPAD (UNIFORM (1, 5, RANDOM ()), 3, '0') AS SALES_REP_ID,
      DATEADD (DAY, - UNIFORM (1, 365, RANDOM ()), CURRENT_DATE) AS REVIEW_DATE,
      UNIFORM (1, 5, RANDOM ()) AS RATING,
      AI_COMPLETE (
        'mistral-large2',
        CONCAT (
          CASE
            UNIFORM (1, 5, RANDOM ())
            WHEN 1 THEN '保険の営業マンの説明の分かりやすさについて、'
            WHEN 2 THEN '保険の営業マンの対応の迅速さについて、'
            WHEN 3 THEN '保険の営業マンのフォローアップについて、'
            WHEN 4 THEN '保険の営業マンの提案内容について、'
            ELSE '保険の営業マンの親身な対応について、'
          END,
          CASE
            WHEN UNIFORM (1, 5, RANDOM ()) = 5 THEN 'とても満足した'
            WHEN UNIFORM (1, 5, RANDOM ()) = 1 THEN '不満が残る'
            ELSE '普通の'
          END,
          'レビューを具体的に1文で書いてください。'
        )
      ) AS REVIEW_COMMENT
    FROM
      TABLE (GENERATOR (ROWCOUNT => 100))
  )
SELECT
  *
FROM
  SAMPLE_DATA;
  /* 作成されたデータの確認 */
SELECT
  *
FROM
  LLM_FOR_BEGINNERS_DB.PUBLIC.INSURANCE_REVIEWS
ORDER BY
  REVIEW_DATE DESC
LIMIT
  5;

In [ ]:
CREATE OR REPLACE
TABLE LLM_FOR_BEGINNERS_DB.PUBLIC.INSURANCE_REVIEWS_ADVICE AS
SELECT *,
    AI_COMPLETE('mistral-large2',
    CONCAT('あなたは保険営業のトップ営業です。次のレビュー文から,次の営業活動にどう活かすべきか教えてください <review>', REVIEW_COMMENT, '</review>')) as AI_ADVICE
FROM LLM_FOR_BEGINNERS_DB.PUBLIC.INSURANCE_REVIEWS;

In [ ]:
df = session.table("LLM_FOR_BEGINNERS_DB.PUBLIC.INSURANCE_REVIEWS_ADVICE")

df = df.collect()
num = st.slider("レビュー番号",0,df.__len__())

review_comment = df[num]['REVIEW_COMMENT']
ai_advice = df[num]['AI_ADVICE']

st.write("### レビューコメント")
st.write(review_comment.strip('"').replace('\\n', '\n'))

st.write("### AIのアドバイス")
st.write(ai_advice.strip('"').replace('\\n', '\n'))

### 1-1-4: AI_COMPLETE 構造化出力
![IMAGE](https://lh3.googleusercontent.com/pw/AP1GczOmD3XpJPMIwT6rzmXRWDK_wDZuiY5Dr4GbX2OVNNsTC7IlLyJpYuQFUkRFR-fWOanoMO0xCfV6XenwjdufIr9n-2-kNlLC_T0Ic7XAQFRPXuXvxP2GG2rkDfbeOPhuiFqGFv71AXQwBrRInXbLfAVT=w3368-h1888-s-no-gm?authuser=0)
- 出力をJSON形式にすることで, 後続処理で利用可能に

In [ ]:
CREATE OR REPLACE
TABLE LLM_FOR_BEGINNERS_DB.PUBLIC.INSURANCE_REVIEWS_ADVICE_ACTIONS AS
SELECT *,
    AI_COMPLETE(
        model => 'mistral-large2',
        prompt => CONCAT('あなたは保険営業のトップ営業です。
            次のレビュー文から,次の営業活動にどう活かすべきかKeep/Problem/Tryのアクションプランをそれぞれ教えてください, なければ「特になし」と表現してください <review>',REVIEW_COMMENT, '</review> 出力の内容として, action_nameはアクションの概要を,action_details_decriptionにはアクションの詳細を,Typeは,Keep,Problem,Tryの3種類から選択してください'),
        response_format => {
            'type': 'json',
                'schema': { 
                    'type': 'object',
                    'properties': {
                        'action_plans': {
                            'type': 'array',
                            'items': {
                                'type': 'object',
                                'properties': {
                                    'action_name': {'type': 'string'},
                                    'type': {'type': 'string'},
                                    'action_details_decription': {'type': 'string'}
                                },
                                'required': ['action_name','type', 'action_details_decription']
                             }
                         }
                     }
                 }
        }
    ) as ACTION_PLANS
FROM LLM_FOR_BEGINNERS_DB.PUBLIC.INSURANCE_REVIEWS_ADVICE;

SELECT review_comment, action_plans
FROM LLM_FOR_BEGINNERS_DB.PUBLIC.INSURANCE_REVIEWS_ADVICE_ACTIONS LIMIT 10;

In [ ]:
import json

df = session.table("LLM_FOR_BEGINNERS_DB.PUBLIC.INSURANCE_REVIEWS_ADVICE_ACTIONS")

df = df.collect()
num = st.slider("レビュー番号",0,100)

review_comment = df[num]['REVIEW_COMMENT']
ai_advice = df[num]['AI_ADVICE']
action_plans = df[num]['ACTION_PLANS']

st.write("### レビューコメント")
st.write(review_comment.strip('"').replace('\\n', '\n'))

st.write("### AIのアドバイス")
st.write(ai_advice.strip('"').replace('\\n', '\n'))

st.write("### アクションプラン")
# JSON文字列をPythonの辞書に変換
data = json.loads(action_plans)

# action_plansリストを取得
action_plans = data.get("action_plans", [])

# 分解して表示
for i, action in enumerate(action_plans, start=1):
    st.write(f"#### アクションプラン{i} ({action.get('type')}): {action.get('action_name')}")
    st.write(f" - {action.get('action_details_decription')}")


### 1-2. SENTIMENT
![IMAGE](https://lh3.googleusercontent.com/pw/AP1GczPGgkdTfDlgL3uWH3V3vom9lECFrQkIMDZG0QqxMlGwaAcqYMgXVpyCTeC5sR0Tu8SHCP_QbXnqgmo9DKocLmystfhMvNHsEHYVQrLD2PHxo61z5ODIlxOmV8uOHP9eAHgDWTiNrXGECPpdFU_BIrMy=w3366-h1888-s-no-gm?authuser=0)

![IMAGE](https://lh3.googleusercontent.com/pw/AP1GczOiN4KxfGjyvlqf0WOyVsXjAEhwThyRaYmIVKh4siXlTtt2ERYniY6qjBrR3WPtOkRlJNZrvJ8_0UtLsJPkK2UbO5lVGd9bXzkobJIvQrewEDD3OqWB3xJXZmfMrHB9iBSjV-9QmS9NPcxPeckozk_u=w3366-h1888-s-no-gm?authuser=0)

In [ ]:
CREATE OR REPLACE TABLE LLM_FOR_BEGINNERS_DB.PUBLIC.INSURANCE_REVIEWS_SENTIMENT AS
SELECT *, 
       SNOWFLAKE.CORTEX.SENTIMENT(
            SNOWFLAKE.CORTEX.TRANSLATE(review_comment ,'ja','en')
        ) as sentiment_score,
        CASE
          WHEN sentiment_score >= 0.5 THEN 'Positive'
          WHEN sentiment_score >= -0.5 THEN 'Neutral'
          ELSE 'Negative'
        END AS sentiment
FROM LLM_FOR_BEGINNERS_DB.PUBLIC.INSURANCE_REVIEWS_ADVICE_ACTIONS;

SELECT review_comment, sentiment, sentiment_score 
FROM LLM_FOR_BEGINNERS_DB.PUBLIC.INSURANCE_REVIEWS_SENTIMENT;

In [ ]:
-- ネガティブなコメントのみを抽出

SELECT review_comment, sentiment, sentiment_score 
FROM LLM_FOR_BEGINNERS_DB.PUBLIC.INSURANCE_REVIEWS_SENTIMENT
WHERE sentiment = 'Negative';

### 1-3. AI_FILTER

![IMAGE](https://lh3.googleusercontent.com/pw/AP1GczMf3eI8zbJy3V1CoUahPvtAT26Fwbb8h73jOxGYn3UpcGPl5iWcahKke8c74d99bnkS2fPtpAaivH0LCo3d2QuLKRnoFKtQCVsfcWfzkrkRME7jTzAbs5uoB5gh8jjHr_NdJmJehuBbubAFAiVZXS4J=w3366-h1888-s-no-gm?authuser=0)

In [ ]:
CREATE OR REPLACE
TABLE LLM_FOR_BEGINNERS_DB.PUBLIC.INSURANCE_NEGATIVE_REVIEWS AS
SELECT *
FROM LLM_FOR_BEGINNERS_DB.PUBLIC.INSURANCE_REVIEWS_SENTIMENT
WHERE AI_FILTER(CONCAT('ネガティブなコメントが含まれるレビュー: ', REVIEW_COMMENT));

SELECT review_comment, sentiment, sentiment_score 
FROM LLM_FOR_BEGINNERS_DB.PUBLIC.INSURANCE_NEGATIVE_REVIEWS;

In [ ]:
df = session.table("LLM_FOR_BEGINNERS_DB.PUBLIC.INSURANCE_NEGATIVE_REVIEWS")

df = df.collect()
num = st.slider("レビュー番号",0,df.__len__())

review_comment = df[num]['REVIEW_COMMENT']
ai_advice = df[num]['AI_ADVICE']
action_plans = df[num]['ACTION_PLANS']

st.write("### レビューコメント")
st.write(review_comment.strip('"').replace('\\n', '\n'))


col1, col2 = st.columns(2)
with col1:
    st.write("### AIのアドバイス")
    st.write(ai_advice.strip('"').replace('\\n', '\n'))
with col2:
    st.write("### アクションプラン")
    # JSON文字列をPythonの辞書に変換
    data = json.loads(action_plans)
    
    # action_plansリストを取得してtypeで並び替え
    action_plans = sorted(data.get("action_plans", []), key=lambda x: x.get('type', ''))
    
    
    # 分解して表示
    for i, action in enumerate(action_plans, start=1):
        st.write(f"#### アクションプラン{i} ({action.get('type')}): {action.get('action_name')}")
        st.write(f" - {action.get('action_details_decription')}")


### 1-4. AI_SUMMARIZE_AGG / AI_AGG

![IMAGE](https://lh3.googleusercontent.com/pw/AP1GczPYQJQ-57pxftQe4XSa5481KbXu1rhBljpVPyuBB9_66r-o-DpfkCJXbkB7EzlW3o-a-afqPj0idNQDWOzPP03aHL7vZRoHY5cb6vUUcCi80crk0hZfn4sd8qgiK-LSwMlOuYBDIbYdY0OwgfOZ2MRR=w3366-h1888-s-no-gm?authuser=0)

![IMAGE](https://lh3.googleusercontent.com/pw/AP1GczP7ZZ1CEjRkf6GNfOXbi4mOMMi0yyyvAmBXN0AGidK0jid_3-XwqGBc2W1JaSk6iF7yH5mWA41Hr29kk2Wnaw57xXpSGHRbnQO5XZ2mFAUXgksq8mdpF9S3edVAew8SvFOdz0a1PwAQlfEHJ1i96BOT=w3366-h1888-s-no-gm?authuser=0)

In [ ]:
CREATE OR REPLACE TABLE LLM_FOR_BEGINNERS_DB.PUBLIC.INSURANCE_NEGATIVE_REVIEWS_SUMMARIZE AS
SELECT SALES_REP_ID,
       min(review_date) as min_review_date,
       max(review_date) as max_review_date,
       count(*) as review_count,
       avg(sentiment_score) as avg_sentiment_score,
       AI_SUMMARIZE_AGG(review_comment) as summarize,
       AI_AGG(review_comment, 'レビュー内容から,業務改善のアクションプランを日本語で教えてください') as actions
FROM LLM_FOR_BEGINNERS_DB.PUBLIC.INSURANCE_NEGATIVE_REVIEWS_ADVICE
GROUP BY SALES_REP_ID
ORDER BY SALES_REP_ID
;

SELECT *
FROM LLM_FOR_BEGINNERS_DB.PUBLIC.INSURANCE_NEGATIVE_REVIEWS_SUMMARIZE;

In [ ]:
import streamlit as st
import pandas as pd
from snowflake.snowpark.context import get_active_session

# セッションの取得
session = get_active_session()

# データの取得
df = session.table('LLM_FOR_BEGINNERS_DB.PUBLIC.INSURANCE_NEGATIVE_REVIEWS_SUMMARIZE').to_pandas()

# タイトル
st.title("ネガティブレビュー分析ダッシュボード")

# 営業マンID選択
sales_rep_id = st.selectbox(
    "営業マンIDを選択してください",
    options=sorted(df['SALES_REP_ID'].unique())
)

# 選択された営業マンのデータを取得
selected_data = df[df['SALES_REP_ID'] == sales_rep_id].iloc[0]

# メトリクスの表示
col1, col2 = st.columns(2)
with col1:
    st.metric("レビュー件数", selected_data['REVIEW_COUNT'])
with col2:
    st.metric("平均センチメントスコア", f"{selected_data['AVG_SENTIMENT_SCORE']:.3f}")

st.metric("レビュー期間", 
          f"{selected_data['MIN_REVIEW_DATE'].strftime('%Y/%m/%d')} - {selected_data['MAX_REVIEW_DATE'].strftime('%Y/%m/%d')}")

# サマリーの表示
st.header("レビューサマリー")
st.write(selected_data['SUMMARIZE'])

# アクションプランの表示
st.header("推奨アクションプラン")
st.write(selected_data['ACTIONS'])

# 全体データの表示（折りたたみ可能）
with st.expander("全データを表示"):
    st.dataframe(
        df.style.format({
            'AVG_SENTIMENT_SCORE': '{:.3f}',
            'MIN_REVIEW_DATE': '{:%Y/%m/%d}',
            'MAX_REVIEW_DATE': '{:%Y/%m/%d}'
        })
    )


### 1-5. AI_SIMILARITY

![IMAGE](https://lh3.googleusercontent.com/pw/AP1GczOlSon-iliWn2SC3rUWxhsRxgTdznhcwX0WExCkjozWCNEyQOTLthDEnue_I6f6B9T5q8yLnuTz0xpoBflIsZAi9vK-t2NqiYbxNk3JCnV-oFfw0iy3CN4lcyvRi_h3X-ejjI2HsJqItgIkHvJft1us=w3366-h1888-s-no-gm?authuser=0)

In [ ]:
CREATE OR REPLACE
TABLE LLM_FOR_BEGINNERS_DB.PUBLIC.SALES_ACTION_PLAN_SIM AS
SELECT
  a.SALES_REP_ID AS SALES_REP_ID_1,
  b.SALES_REP_ID AS SALES_REP_ID_2,
  AI_SIMILARITY (a.KPT, b.KPT) AS SIMILARITY_SCORE
FROM
  LLM_FOR_BEGINNERS_DB.PUBLIC.SALES_ACTION_PLAN AS a
  CROSS JOIN LLM_FOR_BEGINNERS_DB.PUBLIC.SALES_ACTION_PLAN AS b
WHERE
  a.SALES_REP_ID < b.SALES_REP_ID
  /* 重複を避けるため */
ORDER BY
  a.SALES_REP_ID ASC,SIMILARITY_SCORE DESC ;

In [ ]:
import streamlit as st
import pandas as pd
import altair as alt
from snowflake.snowpark.context import get_active_session

# セッションの取得
session = get_active_session()

# データの取得
df = session.table('LLM_FOR_BEGINNERS_DB.PUBLIC.SALES_ACTION_PLAN_SIM').to_pandas()

# 既存のペアに自分自身との類似度(1.0)を追加
unique_ids = sorted(set(df['SALES_REP_ID_1'].unique()) | set(df['SALES_REP_ID_2'].unique()))
self_pairs = pd.DataFrame({
    'SALES_REP_ID_1': unique_ids,
    'SALES_REP_ID_2': unique_ids,
    'SIMILARITY_SCORE': 1.0
})

# 全てのデータを結合
df = pd.concat([df, self_pairs])

# 対称行列を作成するために、ID_1とID_2を入れ替えたデータも追加
reversed_pairs = df[df['SALES_REP_ID_1'] != df['SALES_REP_ID_2']].copy()
reversed_pairs.columns = ['SALES_REP_ID_2', 'SALES_REP_ID_1', 'SIMILARITY_SCORE']
df = pd.concat([df, reversed_pairs])

# 重複を削除
df = df.drop_duplicates()

# ヒートマップの作成
heatmap = alt.Chart(df).mark_rect().encode(
    x=alt.X('SALES_REP_ID_1:N', title='営業マンID', sort=unique_ids),
    y=alt.Y('SALES_REP_ID_2:N', title='営業マンID', sort=unique_ids),
    color=alt.Color('SIMILARITY_SCORE:Q', 
                    scale=alt.Scale(scheme='viridis'),
                    title='類似度スコア')
).properties(
    title='営業マン間のアクションプラン類似度',
    width=500,
    height=400
)

# Streamlitで表示
st.altair_chart(heatmap, use_container_width=True)
